In [1]:
import pandas as pd
import numpy as np

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from scipy.stats import pearsonr

In [2]:
train = pd.read_parquet("../data/train_clean_v3.parquet").reset_index(drop=True)
test = pd.read_parquet("../data/test_clean_v3.parquet").reset_index(drop=True)
sample=pd.read_csv("../data/preds/sample_submission.csv")

### Selected features based on correlation and LightGBM feat importance

In [4]:
selected_features = [
    "X863", "X856", "X344", "X598", "X862", "X385", "X852", "X603", "X860", "X674",
    "X415", "X345", "X137", "X855", "X174", "X302", "X178", "X532", "X168", "X612",
    "X888","X421","X333",
    "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"
]


train= train[selected_features + ["label"]]
test= test[selected_features]

NameError: name 'train' is not defined

In [ ]:
print("Train=",train.shape)
print("Test=",test.shape)
print("Sample=",sample.shape)

Train= (525887, 29)
Test= (538150, 28)
Sample= (538150, 2)


In [ ]:
RMV = ["label"]
FEATURES = [c for c in train.columns if not c in RMV]
print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")

There are 28 FEATURES: ['X863', 'X856', 'X344', 'X598', 'X862', 'X385', 'X852', 'X603', 'X860', 'X674', 'X415', 'X345', 'X137', 'X855', 'X174', 'X302', 'X178', 'X532', 'X168', 'X612', 'X888', 'X421', 'X333', 'bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']


### KFold Cross-Validation

In [6]:
FOLDS = 5
kf = KFold(n_splits=FOLDS)

oof_preds = np.zeros(len(train))
test_preds = np.zeros(len(test))

xgb_params = {
    "tree_method": "gpu_hist",
    "predictor": "gpu_predictor",
    "gpu_id": 0,
    "colsample_bylevel": 0.4778015829774066,
    "colsample_bynode": 0.362764358742407,
    "colsample_bytree": 0.7107423488010493,
    "gamma": 1.7094857725240398,
    "learning_rate": 0.02213323588455387,
    "max_depth": 20,
    "max_leaves": 12,
    "min_child_weight": 16,
    "n_estimators": 1667,
    "n_jobs": -1,
    "random_state": 42,
    "reg_alpha": 39.352415706891264,
    "reg_lambda": 75.44843704068275,
    "subsample": 0.06566669853471274,
    "verbosity": 0
}

for i, (train_idx, valid_idx) in enumerate(kf.split(train)):
    print("#" * 25)
    print(f"### Fold {i + 1}")
    print("#" * 25)

    X_train = train.iloc[train_idx][FEATURES]
    y_train = train.iloc[train_idx]["label"]
    X_valid = train.iloc[valid_idx][FEATURES]
    y_valid = train.iloc[valid_idx]["label"]
    X_test = test[FEATURES]

    model = XGBRegressor(**xgb_params)

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=200
    )

    oof_preds[valid_idx] = model.predict(X_valid)
    test_preds += model.predict(X_test)

pearson_score = pearsonr(train["label"], oof_preds)[0]
print("Final Pearson Correlation = ", pearson_score)

test_preds /= FOLDS

#########################
### Fold 1
#########################
[0]	validation_0-rmse:1.00931
[200]	validation_0-rmse:1.00105
[400]	validation_0-rmse:0.99837
[600]	validation_0-rmse:0.99843
[800]	validation_0-rmse:0.99861
[1000]	validation_0-rmse:0.99905
[1200]	validation_0-rmse:0.99926
[1400]	validation_0-rmse:0.99968
[1600]	validation_0-rmse:1.00014
[1666]	validation_0-rmse:1.00027
#########################
### Fold 2
#########################
[0]	validation_0-rmse:0.94909
[200]	validation_0-rmse:0.94529
[400]	validation_0-rmse:0.94552
[600]	validation_0-rmse:0.94570
[800]	validation_0-rmse:0.94634
[1000]	validation_0-rmse:0.94682
[1200]	validation_0-rmse:0.94666
[1400]	validation_0-rmse:0.94698
[1600]	validation_0-rmse:0.94716
[1666]	validation_0-rmse:0.94719
#########################
### Fold 3
#########################
[0]	validation_0-rmse:1.02343
[200]	validation_0-rmse:1.02377
[400]	validation_0-rmse:1.02550
[600]	validation_0-rmse:1.02570
[800]	validation_0-rmse:1.02609
[1000]	

In [8]:
sample["prediction"] = test_preds
sample.to_csv("../data/preds/test_xgboost.csv", index=False)
sample.head()

,ID,prediction
0,1,0.205636
1,2,0.017193
2,3,0.194899
3,4,0.033934
4,5,0.127066
